## 프로보노 공모전에서 앱용 데이터로 변환
- 크롤링에서 공공데이터 API를 사용

### 공공데이터 API로 데이터 가져오기

In [1]:
import requests
import pprint
import json
import pandas as pd

url = 'https://api.odcloud.kr/api/15083323/v1/uddi:541ea710-74e8-4010-b69e-e3cb8b38b3fc?page=1&perPage=400&serviceKey=LL%2FWwEZVOVH0Zl%2BA4ukk09hBFr06qaWdqnjIm%2ByV7WO1O92U5Dj4IcgEHK4Od7pdTibXe662u1oLZxMmKPwIyQ%3D%3D'

response = requests.get(url)
contents = response.text
json_ob = json.loads(contents)

current_count = json_ob['currentCount']
welfares = json_ob['data']

welfares_origin_df= pd.json_normalize(welfares)

In [2]:
welfares_df = welfares_origin_df[['서비스아이디', '서비스명', '서비스요약', '대표문의', '사이트']].copy()

In [3]:
welfares_df.head()

,서비스아이디,서비스명,서비스요약,대표문의,사이트
0,10,(북한이탈주민) 탈북청소년 교육지원,탈북청소년의 학교생활 적응 지원을 통해 대한민국의 건강한 구성원으로 자립할 수 있도...,남북하나재단 콜센터 1577-6635,북한이탈주민지원재단 교육지원부 www.koreahana.or.kr 통일부 정착지원과...
1,18,(특수교육대상자) 치료지원서비스,특수교육대상자의 효과적인 교육을 위해 다양한 서비스를 제공합니다.,교육부 민원콜센터 02-6222-6060,교육부 http://www.moe.go.kr
2,266,6.25자녀수당,6.25 전쟁 중 전사하거나 순직한 전몰군경 혹은 순직군경 자녀의 생활안정과 복지향...,보훈상담센터 1577-0606,국가보훈처 http://www.mpva.go.kr/
3,19,WEE 클래스 상담지원,"초중고 학교부적응 학생 및 위기학생, 일반학생에 대한 학교생활 적응 조력 및 치유를...","학교안전통합시스템 043)5309-182, 184",학생안전통합시스템 www.wee.go.kr
4,357,가사·간병 방문 지원사업,일상생활이 어려운 저소득층 가정에 간병·가사 서비스를 지원하여 취약계층의 생활 안정...,보건복지콜센터 129,사회서비스바우처 http://www.socialservice.or.kr 보건복지부 ...


In [4]:
welfares_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   서비스아이디  357 non-null    int64 
 1   서비스명    357 non-null    object
 2   서비스요약   357 non-null    object
 3   대표문의    356 non-null    object
 4   사이트     317 non-null    object
dtypes: int64(1), object(4)
memory usage: 14.1+ KB


### 전처리 수행

In [5]:
import re
## summary, who, what, criteria 등 모두 해당하기
def preprocess_summary(text):
    if not text:
        return '해당없음'
    text=re.sub(r'\t', '', text)
    text=re.sub(r'^(\n)+|(\n)+$','', text)
    text=re.sub(r'<BR />', '\n', text)
    return text

## call 과 site의 경우
def preprocess_call(text):
    if not text:
        return '해당없음'
    text=re.sub('\t', '', text)
    text=re.sub(r'^(\n)+|(\n)+$','', text)
    text=re.sub(r'\n', ' ', text)
    text=re.sub(r'☎', '', text)
    return text

In [6]:
welfares_df['서비스요약'] = welfares_df['서비스요약'].map(preprocess_summary)
welfares_df[['대표문의', '사이트']] = welfares_df[['대표문의', '사이트']].applymap(preprocess_call)
welfares_df['서비스아이디']=welfares_df['서비스아이디'].astype(str) # sql에 넣기 위한 작업

### 기존 DB와 API 사이의 공통적인 복지 찾기

In [7]:
def preprocess_delete_all(text):
    return re.sub('[^A-Za-z0-9가-힣]', '', text)

In [203]:
import mysql.connector
import config

cnxn = mysql.connector.connect(**config.DATABASE_CONFIG)
cursor = cnxn.cursor(prepared=True)

In [204]:
query = ("SELECT welfare_id, title FROM welfare")
cursor.execute(query)

old_welfares = []
for old_welfare in cursor:
    old_welfares.append([old_welfare[0], preprocess_delete_all(old_welfare[1])])
old_welfares_df = pd.DataFrame(old_welfares, columns=['서비스아이디', '서비스명'])
old_welfares_df['서비스아이디']=old_welfares_df['서비스아이디'].astype(str) # sql에 넣기 위한 작업

### 기존 db 복지정보에서 가지고 있는 복지 정보

In [205]:
new_welfares_df = welfares_df[['서비스아이디', '서비스명']].copy()
new_welfares_df['서비스명'] = new_welfares_df['서비스명'].map(preprocess_delete_all)

In [206]:
inner_welfares_df = pd.merge(old_welfares_df, new_welfares_df, how='inner', on='서비스명')

In [207]:
inner_welfares_df.head()

,서비스아이디_x,서비스명,서비스아이디_y
0,0,청소년한부모자립지원,155
1,1,긴급복지생계지원,303
2,2,교육복지우선지원사업,20
3,3,방과후보육료지원,294
4,4,취약계층환경성질환예방사업,90


In [208]:
inner_welfares_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 351 entries, 0 to 350
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   서비스아이디_x  351 non-null    object
 1   서비스명      351 non-null    object
 2   서비스아이디_y  351 non-null    object
dtypes: object(3)
memory usage: 11.0+ KB


In [209]:
inner_welfares_df.shape

(351, 3)

### 중복되어 있는 복지 정보가 존재함(db 값이 잘못된 경우)

In [210]:
len(inner_welfares_df['서비스아이디_y'])

351

In [211]:
len(set(inner_welfares_df['서비스아이디_y']))

349

In [212]:
from collections import Counter

Counter(inner_welfares_df['서비스아이디_y']).most_common()

[('15281', 2),
 ('129', 2),
 ('155', 1),
 ('303', 1),
 ('20', 1),
 ('294', 1),
 ('90', 1),
 ('58', 1),
 ('216', 1),
 ('352', 1),
 ('12394', 1),
 ('200', 1),
 ('16489', 1),
 ('213', 1),
 ('61', 1),
 ('226', 1),
 ('357', 1),
 ('195', 1),
 ('196', 1),
 ('66', 1),
 ('73', 1),
 ('190', 1),
 ('353', 1),
 ('46', 1),
 ('82', 1),
 ('227', 1),
 ('229', 1),
 ('171', 1),
 ('159', 1),
 ('138', 1),
 ('163', 1),
 ('16440', 1),
 ('143', 1),
 ('223', 1),
 ('133', 1),
 ('17521', 1),
 ('220', 1),
 ('126', 1),
 ('28', 1),
 ('13', 1),
 ('14', 1),
 ('12405', 1),
 ('12392', 1),
 ('110', 1),
 ('12391', 1),
 ('12396', 1),
 ('136', 1),
 ('221', 1),
 ('306', 1),
 ('147', 1),
 ('230', 1),
 ('12395', 1),
 ('282', 1),
 ('16', 1),
 ('165', 1),
 ('16437', 1),
 ('17162', 1),
 ('17520', 1),
 ('2', 1),
 ('341', 1),
 ('144', 1),
 ('318', 1),
 ('335', 1),
 ('52', 1),
 ('191', 1),
 ('241', 1),
 ('158', 1),
 ('16493', 1),
 ('298', 1),
 ('300', 1),
 ('349', 1),
 ('314', 1),
 ('118', 1),
 ('189', 1),
 ('293', 1),
 ('16488', 1

In [213]:
print(inner_welfares_df[inner_welfares_df['서비스아이디_y']=='15281'])
print(inner_welfares_df[inner_welfares_df['서비스아이디_y']=='129' ])

   서비스아이디_x        서비스명 서비스아이디_y
59       59  산림복지서비스이용권    15281
60      399  산림복지서비스이용권    15281
    서비스아이디_x         서비스명 서비스아이디_y
196      201  북한이탈주민의료비지원      129
197      374  북한이탈주민의료비지원      129


In [214]:
inner_welfares_df =inner_welfares_df.drop_duplicates(subset=['서비스아이디_y'], keep='first')
inner_welfares_df.shape

(349, 3)

In [215]:
print(inner_welfares_df[inner_welfares_df['서비스아이디_y']=='15281'])
print(inner_welfares_df[inner_welfares_df['서비스아이디_y']=='129' ])

   서비스아이디_x        서비스명 서비스아이디_y
59       59  산림복지서비스이용권    15281
    서비스아이디_x         서비스명 서비스아이디_y
196      201  북한이탈주민의료비지원      129


### 기존 db 복지정보에서 가지고 있지 않은 새로운 복지정보 

In [216]:
new_welfares_list = new_welfares_df.values.tolist()
old_welfares_list = old_welfares_df['서비스명'].tolist()
diff_welfares = [welfare for welfare in new_welfares_list if welfare[1] not in old_welfares_list]
diff_welfares 

[['21', '다문화탈북학생멘토링'],
 ['33', '독거노인사회관계활성화지원'],
 ['166', '매체활용능력증진및역기능해소청소년인터넷스마트폰과의존치료비지원'],
 ['38', '사회복지종사자상해보험가입지원'],
 ['89', '서민층가스시설개선'],
 ['310', '의료급여의료급여'],
 ['156', '중앙아동보호전문기관운영'],
 ['17163', '최저임금적용제외근로장애인전환지원']]

### mysql에 new_welfare새로운 값 넣기

In [217]:
inner_id_list = inner_welfares_df['서비스아이디_y'].tolist()
sql_new_welfares_df = welfares_df[welfares_df['서비스아이디'].isin(inner_id_list)].copy()

In [218]:
sql_new_welfares_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349 entries, 0 to 356
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   서비스아이디  349 non-null    object
 1   서비스명    349 non-null    object
 2   서비스요약   349 non-null    object
 3   대표문의    349 non-null    object
 4   사이트     349 non-null    object
dtypes: object(5)
memory usage: 16.4+ KB


In [150]:
query = ("""INSERT INTO new_welfare (welfare_id, title, summary, calls, sites) VALUES (%s, %s, %s, %s, %s)""") 
input_data=[tuple(x) for x in sql_new_welfares_df.to_records(index=False)]
cursor.executemany(query, input_data)

cnxn.commit() 

### mysql에 new_welfare_category 넣기

In [219]:
query = ("SELECT welfare_id, category_id FROM welfare_category")
cursor.execute(query)

old_categories = []
for old_category in cursor:
    old_categories.append([old_category[0], old_category[1]])
old_categories_df = pd.DataFrame(old_categories, columns=['서비스아이디', '카테고리'])
old_categories_df = old_categories_df.astype(str)

In [220]:
old_categories_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321 entries, 0 to 1320
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   서비스아이디  1321 non-null   object
 1   카테고리    1321 non-null   object
dtypes: object(2)
memory usage: 20.8+ KB


In [221]:
old_categories_df.drop_duplicates().info() # 중복된 값이 존재하지 않음

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1321 entries, 0 to 1320
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   서비스아이디  1321 non-null   object
 1   카테고리    1321 non-null   object
dtypes: object(2)
memory usage: 31.0+ KB


In [222]:
new_categories_df = pd.merge(old_categories_df, inner_welfares_df, how='inner', left_on='서비스아이디', right_on='서비스아이디_x')

In [223]:
sql_new_categories_df = new_categories_df[['서비스아이디_y', '카테고리']].copy()

In [169]:
query = ("""INSERT INTO new_welfare_category (welfare_id, category_id) VALUES (%s, %s)""") 
input_data=[tuple(x) for x in sql_new_categories_df.to_records(index=False)]
cursor.executemany(query, input_data)

cnxn.commit()

### 8개의 데이터에 대해서 수작업하기

In [224]:
id_list = [id[0] for id in diff_welfares]

In [225]:
addition_welfares_df = welfares_df[welfares_df['서비스아이디'].isin(id_list)].copy()

In [226]:
addition_welfares_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 87 to 329
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   서비스아이디  8 non-null      object
 1   서비스명    8 non-null      object
 2   서비스요약   8 non-null      object
 3   대표문의    8 non-null      object
 4   사이트     8 non-null      object
dtypes: object(5)
memory usage: 384.0+ bytes


In [228]:
query = ("""INSERT INTO new_welfare (welfare_id, title, summary, calls, sites) VALUES (%s, %s, %s, %s, %s)""") 
input_data=[tuple(x) for x in addition_welfares_df.to_records(index=False)]
cursor.executemany(query, input_data)문

cnxn.commit() 

### KSBERT 학습을 위한 csv파일로 만들기

In [9]:
welfares_for_embedding = welfares_df[['서비스아이디', '서비스명', '서비스요약']].copy()

In [12]:
welfares_for_embedding['full'] = welfares_df['서비스명']+' '+welfares_df['서비스요약']
welfares_for_embedding.rename(columns = {'서비스아이디':'welfare_id','서비스명':'title', '서비스요약':'summary'})

In [14]:
welfares_for_embedding.to_csv("welfare_for_embedding.csv", index=False, encoding="utf-8-sig")